### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [6]:
# user_mat, movie_mat = # use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 42.525363
2 		 14.156396
3 		 4.036794
4 		 2.827478
5 		 2.693330
6 		 2.654928
7 		 2.629604
8 		 2.603792
9 		 2.574742
10 		 2.541837


In [7]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.24136455  8.59052305 10.46702502 10.35073655]
 [ 8.75692591  7.313922    8.89506337  8.95026497]
 [ 7.69159196  7.1580791   8.47517169  7.56216056]
 [ 9.42734319  8.44947603 10.06876949  9.52302427]
 [ 8.69211257  7.25524346  8.75480111  8.74976138]
 [ 6.98378842  5.95782333  6.9926277   6.98195787]
 [ 9.10015198  8.1981121   9.7316135   9.13394127]
 [ 8.36323218  7.02582985  8.40977465  8.36966809]
 [ 8.2064561   7.68841578  9.18565773  8.2666235 ]
 [ 7.64803204  6.74567201  8.19808433  7.57550509]
 [ 8.93519303  7.75041775  9.1046326   9.02862321]
 [ 9.58214278  8.64052041 10.17764853  9.34276954]
 [ 9.33642536  8.02081396  9.60555667  9.37674596]
 [ 6.57692969  5.89620149  6.9536623   6.58074578]
 [ 9.73396448  8.3353466  10.13217681  9.94882931]
 [ 9.64506525  8.54540034 10.31257519  9.7153023 ]
 [ 8.27181759  7.30564224  8.70004598  8.28577222]
 [ 7.39644659  5.95289685  7.06583964  7.42242344]
 [ 8.38534511  7.38499441  8.91834437  8.24111018]
 [ 8.09437802  7.36079609  8.59

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [8]:
# user_mat, movie_mat = #use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 41.690218
2 		 13.565667
3 		 3.837661
4 		 2.729135
5 		 2.602917
6 		 2.554072
7 		 2.517253
8 		 2.481347
9 		 2.443560
10 		 2.403113
11 		 2.359714
12 		 2.313194
13 		 2.263422
14 		 2.210285
15 		 2.153673
16 		 2.093471
17 		 2.029547
18 		 1.961748
19 		 1.889911
20 		 1.813876
21 		 1.733523
22 		 1.648819
23 		 1.559877
24 		 1.467021
25 		 1.370837
26 		 1.272208
27 		 1.172308
28 		 1.072554
29 		 0.974507
30 		 0.879744
31 		 0.789706
32 		 0.705571
33 		 0.628154
34 		 0.557873
35 		 0.494766
36 		 0.438558
37 		 0.388752
38 		 0.344725
39 		 0.305813
40 		 0.271373
41 		 0.240818
42 		 0.213643
43 		 0.189418
44 		 0.167790
45 		 0.148465
46 		 0.131199
47 		 0.115786
48 		 0.102047
49 		 0.089823
50 		 0.078972
51 		 0.069361
52 		 0.060869
53 		 0.053383
54 		 0.046797
55 		 0.041014
56 		 0.035945
57 		 0.031506
58 		 0.027624
59 		 0.024231
60 		 0.021267
61 		 0.018678
62 		 0.016416
63 		 0.014440
64 		

In [9]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.99999279  9.99999672  9.99999828 10.00000762]
 [10.00000476  4.00000233  9.00000128  9.99999492]
 [ 8.00000012  9.00000014 10.00000012  4.99999987]
 [ 9.00000204  8.00000099 10.00000055  9.99999783]
 [ 9.99999925  4.9999996   8.99999978  9.00000081]
 [ 5.99999619  3.99999823  9.99999906  6.00000406]
 [ 9.00000293  8.00000147 10.0000008   8.99999685]
 [ 9.99999771  4.99999895  8.99999944  8.00000242]
 [ 7.00000377  8.00000182 10.00000098  7.99999597]
 [ 8.99999817  4.99999914  8.99999954  7.00000195]
 [ 9.00000568  8.00000264 10.0000014   7.99999397]
 [ 8.99999801  9.99999901  9.99999944  9.00000212]
 [ 9.99999926  8.99999964  9.9999998   8.00000078]
 [ 5.00000301  8.00000139  5.00000073  7.99999677]
 [10.00000001  7.99999993  9.99999994  9.99999999]
 [ 9.00000055  9.00000022 10.00000008  9.99999938]
 [ 9.000002    8.00000085  8.00000043  7.99999789]
 [ 9.99999852  7.99999928  0.99999958 10.00000152]
 [ 4.99999684  5.99999847  9.99999915 10.00000333]
 [ 7.99999904  6.99999954  9.99

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [10]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [11]:
# run SVD on the matrix with the missing value
# user_mat, movie_mat = #use your function with 4 latent features, lr of 0.005 and 250 iterations

user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 44.840418
2 		 16.121599
3 		 4.676288
4 		 2.968230
5 		 2.718181
6 		 2.627391
7 		 2.565420
8 		 2.507950
9 		 2.447835
10 		 2.382473
11 		 2.310845
12 		 2.232722
13 		 2.148462
14 		 2.058964
15 		 1.965626
16 		 1.870258
17 		 1.774926
18 		 1.681748
19 		 1.592654
20 		 1.509179
21 		 1.432319
22 		 1.362481
23 		 1.299537
24 		 1.242948
25 		 1.191922
26 		 1.145570
27 		 1.103036
28 		 1.063577
29 		 1.026612
30 		 0.991722
31 		 0.958643
32 		 0.927235
33 		 0.897448
34 		 0.869290
35 		 0.842800
36 		 0.818019
37 		 0.794977
38 		 0.773682
39 		 0.754111
40 		 0.736212
41 		 0.719909
42 		 0.705101
43 		 0.691675
44 		 0.679508
45 		 0.668473
46 		 0.658448
47 		 0.649316
48 		 0.640967
49 		 0.633304
50 		 0.626239
51 		 0.619695
52 		 0.613607
53 		 0.607916
54 		 0.602575
55 		 0.597543
56 		 0.592785
57 		 0.588273
58 		 0.583981
59 		 0.579890
60 		 0.575981
61 		 0.572239
62 		 0.568653
63 		 0.565209
64 		

In [12]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 11.197593189636631:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [13]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20) #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.647892
2 		 10.837921
3 		 7.417975
4 		 5.721170
5 		 4.651038
6 		 3.902899
7 		 3.347265
8 		 2.917404
9 		 2.575033
10 		 2.296476
11 		 2.066224
12 		 1.873618
13 		 1.711014
14 		 1.572708
15 		 1.454310
16 		 1.352362
17 		 1.264098
18 		 1.187287
19 		 1.120116
20 		 1.061101


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [14]:
# Replace each of the comments below with the correct values
# num_ratings = # How many actual ratings exist in first_1000_users
# print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
# print()

# ratings_for_missing = # How many ratings did we make for user-movie pairs that didn't actually have ratings
# print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [15]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
